# Ejercicio 6
Instrucciones: Produzca el gráfico que se muestra en el codigo de python en Julia. Puede utilizar el paquete Plots o el paquete Makie.
1. Presente su proyecto siguiendo las directrices de presentación
2. Asegúrese de incluir el código y la(s) figura(s).

In [ ]:
using Plots
using Graphs
using Random
using GraphPlot
using ColorSchemes
using Colors
using Distributions
using Makie
using GLMakie

### Definición de Grafo Geométrico Aleatorio

Un **grafo geométrico aleatorio** es un grafo en el que los nodos son distribuidos aleatoriamente en un espacio $\mathbb{R}^d$ (comúnmente en el plano $\mathbb{R}^2$). Luego, se conecta dos nodos $i$ y $j$ si la **distancia** $d(i, j)$ entre ellos es menor o igual a un **radio** $r$. Formalmente, un grafo geométrico aleatorio se define como sigue:

- Los nodos $i$ son distribuidos aleatoriamente en el espacio (por ejemplo, en el plano $\mathbb{R}^2$).
- Se coloca una **arista** entre los nodos $i$ y $j$ si la distancia entre ellos es menor o igual que el radio $r$:

$$
d(i, j) \leq r
$$

donde $d(i, j)$ es la distancia entre los nodos $i$ y $j$, generalmente medida mediante la distancia euclidiana.

1. > Philippe Jacquet, Wolfgang Szpankowski. *Random Geometric Graph: Some recent developments and applications*.

2. > Bollobás, Béla. *Random Graphs*. Cambridge University Press.

In [ ]:
function random_geometric_graph(n::Int, radius::Float64)
    # Generar posiciones aleatorias para los nodos en el plano
    positions = [(Random.rand(Uniform(0,1)), Random.rand(Uniform(0,1))) for _ in 1:n]

    # Crear un diccionario con las posiciones
    pos_dict = Dict(i => positions[i] for i in 1:n)

    # Crear un grafo vacío con n nodos
    g = Graphs.SimpleGraph(n)

    # Conectar nodos si cumplen con la condición de distancia
    for i in 1:n
        for j in i+1:n
            # Calculando la distancia euclidiana entre nodos
            if sqrt((positions[i][1] - positions[j][1])^2 + (positions[i][2] - positions[j][2])^2) <= radius
                Graphs.add_edge!(g, i, j)
            end
        end
    end

    return g, pos_dict
end

random_geometric_graph (generic function with 1 method)

Con la función anterior se crea una grafico y n posiciones de los nodos. Posteriormente definimos el número de nodos y el radio dentro del cual se conectaran los nodos cercanos. Tambien determinamos el nodo central para que podamos hacer un degradado del color mientras má lejos esten del nodo central.

In [ ]:
nodos = 200
radio = 0.125

G, posiciones = random_geometric_graph(nodos, radio)

coordenada_x = [v[1] for v in values(posiciones)]
coordenada_y = [v[2] for v in values(posiciones)]

# Determinar el nodo central más cercano a (0.5, 0.5)
nodo_central = argmin([(pos[1] - 0.5)^2 + (pos[2] - 0.5)^2 for pos in values(posiciones)])

# Calcular las distancias desde el nodo central
distancias = Graphs.dijkstra_shortest_paths(G, nodo_central).dists

# Normalizar las distancias para el mapeo de colores
distancia_maxima = maximum(distancias)
distancias_normalizadas = distancias ./ distancia_maxima;

In [ ]:
# Crear la figura para los nodos
fig, ax, scatter_plot = Makie.scatter(coordenada_x, coordenada_y, markersize=20)

hidespines!(ax)
ax.xticklabelsvisible = false
ax.yticklabelsvisible = false

# Calcular las distancias desde el nodo central
distancias_central = [sqrt((coordenada_x[i] - coordenada_x[nodo_central])^2 + (coordenada_y[i] - coordenada_y[nodo_central])^2) for i in 1:nodos]

# Normalizar las distancias
distancia_maxima = maximum(distancias_central)
distancias_normalizadas = distancias_central ./ distancia_maxima

# Invertir las distancias normalizadas (para que los nodos más cercanos sean más rojos)
distancias_normalizadas_invertidas = 1 .- distancias_normalizadas

# Crear un mapa de colores
cmapa = ColorSchemes.Reds.colors

# Asignar color a cada nodo según su distancia al nodo central (invertido)
colores_nodos = [cmapa[Int(round(distancia * (length(cmapa) - 1))) + 1] for distancia in distancias_normalizadas_invertidas]

#Gráfico con los nodos con el color degradado
scatter_plot = Makie.scatter!(coordenada_x, coordenada_y, color=colores_nodos, markersize=20)

# Agregar líneas entre los puntos que cumplan la condición
for i in 1:length(coordenada_x)
    for j in i+1:length(coordenada_x)
        distancia = sqrt((coordenada_x[i] - coordenada_x[j])^2 + (coordenada_y[i] - coordenada_y[j])^2)
        if distancia < radio
            # Calcular el color según las distancias de los nodos
            color_rojo = cmapa[Int(round(max(distancias_normalizadas_invertidas[i], distancias_normalizadas_invertidas[j]) * (length(cmapa) - 1))) + 1]
            lines!(ax, [coordenada_x[i], coordenada_x[j]], [coordenada_y[i], coordenada_y[j]], color=color_rojo)
        end
    end
end

# Mostrar el gráfico
fig
